In [ ]:
import numpy as np

In [ ]:
import re
from nltk.corpus import stopwords
import pandas as pd

def preprocess(raw_text):
    if isinstance(raw_text, str):
        # Keep only words and remove punctuation
        cleaned_words = re.findall(r'\b\w+\b', raw_text.lower())

        # Remove stopwords
        stopword_set = set(stopwords.words("english"))
        cleaned_words = [w for w in cleaned_words if w not in stopword_set and w in model]
    else:
        # Handle non-string inputs
        cleaned_words = []

    return cleaned_words


def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(model[word1], model[word2]))

In [ ]:
# Read data from CSV file
import pandas as pd
data = pd.read_csv('hacker_news.csv')

In [ ]:
data

,Question,Manual Answer,Bot Answer,CONTEXT
0,Who were the suspected creators of the Stuxnet...,United States and Israel,The United States and Israel are widely believ...,NaN
1,What was the name of the ransomware used by FIN8?,White Rabbit ransomware,FIN8 has been detected using the White Rabbit ...,NaN
2,Which chips are affected by the ROCA attack?,The ROCA attack affects chips manufactured by ...,Chips manufactured by Infineon as early as 201...,NaN
3,Who were the victims of the cyber espionage op...,Victims included Members of the European Parli...,"Members of the European Parliament, Catalan Pr...",NaN
4,What versions of Junos OS are affected by the ...,All versions of Junos OS on SRX and EX Series,All versions of Junos OS on SRX and EX Series ...,NaN
...,...,...,...,...
94,When did the EleKtra-Leak campaign become active?,active since at least December 2020,The EleKtra-Leak campaign became active on Dec...,https://thehackernews.com/2023/10/elektra-leak...
95,What is the name of the ongoing campaign targe...,EleKtra-Leak.,The name of the campaign is EleKtra-Leak.,https://thehackernews.com/2023/10/elektra-leak...
96,Which web browsers are susceptible to the iLea...,orks against all third-party web browsers avai...,All third-party web browsers available for iOS...,https://thehackernews.com/2023/10/ileakage-new...
97,What optimization mechanism in modern CPUs for...,The iLeakage attack exploits the speculative d...,The iLeakage attack exploits the speculative d...,https://thehackernews.com/2023/10/ileakage-new...


In [ ]:
questions=data['Question'].to_list()

In [ ]:
answers=data['Manual Answer'].to_list()

In [ ]:
bot_answer=data['Bot Answer'].to_list()

In [ ]:
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model





In [ ]:

model = loadGloveModel('glove.6B.300d.txt')


Loading Glove Model
Done. 400000  words loaded!


In [ ]:
from scipy import spatial

In [ ]:
def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)

    vector_1 = vector_1.flatten()
    vector_2 = vector_2.flatten()
    # print(vector_1,vector_2)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')
    return round((1-cosine)*100,2)
def heat_map_matrix_between_two_sentences(s1,s2):
    df = calculate_heat_matrix_for_two_sentences(s1,s2)
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(5,5))
    ax_blue = sns.heatmap(df, cmap="YlGnBu")
    # ax_red = sns.heatmap(df)
    print(cosine_distance_wordembedding_method(s1, s2))
    return ax_blue




In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Keep only 'Question', 'Human Answer', and 'Bot Answer' columns
data = data[['Question', 'Manual Answer', 'Bot Answer']]
# Calculate similarity scores and save them to a new DataFrame
similarity_scores = []
for index, row in data.iterrows():
    s1 = row['Manual Answer']
    s2 = row['Bot Answer']
    # print(s1,'\n',s2,'\n')
    similarity_score = cosine_distance_wordembedding_method(s1, s2)
    similarity_scores.append(similarity_score)

data['S2'] = similarity_scores


Word Embedding method with a cosine distance asses that our two sentences are similar to 77.62 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 70.69 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 73.03 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 98.54 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 86.56 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 61.44 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 23.97 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 100 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 86.27 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 100 %
Word Embedding method with a cosin

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Word Embedding method with a cosine distance asses that our two sentences are similar to 72.52 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 49.25 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 78.41 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 93.15 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 76.59 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 100 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 100 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 100 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 82.7 %
Word Embedding method with a cosine distance asses that our two sentences are similar to 67.98 %
Word Embedding method with a cosine d

In [ ]:
len(data['S2'])

99

In [ ]:
# type(data.iloc[16]['Human Answer'])

In [ ]:
data

,Question,Manual Answer,Bot Answer,S2
0,Who were the suspected creators of the Stuxnet...,United States and Israel,The United States and Israel are widely believ...,77.62
1,What was the name of the ransomware used by FIN8?,White Rabbit ransomware,FIN8 has been detected using the White Rabbit ...,70.69
2,Which chips are affected by the ROCA attack?,The ROCA attack affects chips manufactured by ...,Chips manufactured by Infineon as early as 201...,73.03
3,Who were the victims of the cyber espionage op...,Victims included Members of the European Parli...,"Members of the European Parliament, Catalan Pr...",98.54
4,What versions of Junos OS are affected by the ...,All versions of Junos OS on SRX and EX Series,All versions of Junos OS on SRX and EX Series ...,86.56
...,...,...,...,...
94,When did the EleKtra-Leak campaign become active?,active since at least December 2020,The EleKtra-Leak campaign became active on Dec...,75.69
95,What is the name of the ongoing campaign targe...,EleKtra-Leak.,The name of the campaign is EleKtra-Leak.,70.22
96,Which web browsers are susceptible to the iLea...,orks against all third-party web browsers avai...,All third-party web browsers available for iOS...,86.00
97,What optimization mechanism in modern CPUs for...,The iLeakage attack exploits the speculative d...,The iLeakage attack exploits the speculative d...,51.12


In [ ]:
# Save the DataFrame to a new CSV file
data.to_csv('glovesimilarity_hacknews.csv', index=False)

In [ ]:
data

,Question,Manual Answer,Bot Answer,S2
0,Who were the suspected creators of the Stuxnet...,United States and Israel,The United States and Israel are widely believ...,77.62
1,What was the name of the ransomware used by FIN8?,White Rabbit ransomware,FIN8 has been detected using the White Rabbit ...,70.69
2,Which chips are affected by the ROCA attack?,The ROCA attack affects chips manufactured by ...,Chips manufactured by Infineon as early as 201...,73.03
3,Who were the victims of the cyber espionage op...,Victims included Members of the European Parli...,"Members of the European Parliament, Catalan Pr...",98.54
4,What versions of Junos OS are affected by the ...,All versions of Junos OS on SRX and EX Series,All versions of Junos OS on SRX and EX Series ...,86.56
...,...,...,...,...
94,When did the EleKtra-Leak campaign become active?,active since at least December 2020,The EleKtra-Leak campaign became active on Dec...,75.69
95,What is the name of the ongoing campaign targe...,EleKtra-Leak.,The name of the campaign is EleKtra-Leak.,70.22
96,Which web browsers are susceptible to the iLea...,orks against all third-party web browsers avai...,All third-party web browsers available for iOS...,86.00
97,What optimization mechanism in modern CPUs for...,The iLeakage attack exploits the speculative d...,The iLeakage attack exploits the speculative d...,51.12
